# <div class="alert alert-block alert-info">Работа с рекомендательными системами

## <div class="alert alert-block alert-success">Цель исследования: сформировать вопросы и ответы на основе матрицы предпочтений.

In [186]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np
from itertools import combinations
import seaborn as sns
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

## <a class="anchor" id="1"><div class="alert alert-block alert-success">Предобработка данных

In [187]:
# загрузим данные
user_item_matrix = pd.read_excel('ratingNames.xlsx')
user_item_matrix

,название фильма,Алексей,Алена,Алина,Альбина,Анастасия,Ангелина,Андрей,Анна,Арина,...,Кристина,Майя,Максим,Надежда,Полина,Самира,София,Ульяна,Юлия,Юрий
0,«Дэдпул» (2016),8,7,0,0,0,10,0,8,0,...,10,10,10,0,7,0,0,10,0,7
1,«Бумажный дом» (2019),7,0,0,0,0,10,10,10,0,...,10,10,10,0,0,0,0,7,10,7
2,«Голодные игры» (2012),6,5,0,7,0,10,10,7,0,...,0,10,10,0,10,0,0,8,0,8
3,«Стражи Галактики» (2014),9,8,0,0,0,7,0,10,0,...,10,10,10,0,10,0,0,10,0,10
4,«Интерстеллар» (2014),10,10,0,10,0,10,6,10,0,...,0,10,10,0,10,0,0,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Харли Квинн,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,0,8
89,Слабый герой,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0
90,Мошенники,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0
91,Укрытие,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0


Дана матрица предпочтений, в которой 93 строки (фильма) и 31 колонка (пользователи)

In [188]:
# просматриваем информацию
user_item_matrix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 31 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   название фильма  93 non-null     object
 1   Алексей          93 non-null     int64 
 2   Алена            93 non-null     int64 
 3   Алина            93 non-null     int64 
 4   Альбина          93 non-null     int64 
 5   Анастасия        93 non-null     int64 
 6   Ангелина         93 non-null     int64 
 7   Андрей           93 non-null     int64 
 8   Анна             93 non-null     int64 
 9   Арина            93 non-null     int64 
 10  Артём            93 non-null     int64 
 11  Валерия          93 non-null     int64 
 12  Виктория         93 non-null     int64 
 13  Всеволод         93 non-null     int64 
 14  Георгий          93 non-null     int64 
 15  Дарья            93 non-null     int64 
 16  Дмитрий          93 non-null     int64 
 17  Екатерина        93 non-null     int6

In [189]:
# проверим пропуски
user_item_matrix.isnull().sum()

название фильма    0
Алексей            0
Алена              0
Алина              0
Альбина            0
Анастасия          0
Ангелина           0
Андрей             0
Анна               0
Арина              0
Артём              0
Валерия            0
Виктория           0
Всеволод           0
Георгий            0
Дарья              0
Дмитрий            0
Екатерина          0
Елизавета          0
Игнат              0
Ирина              0
Кристина           0
Майя               0
Максим             0
Надежда            0
Полина             0
Самира             0
София              0
Ульяна             0
Юлия               0
Юрий               0
dtype: int64

In [190]:
# проверим дубликаты
user_item_matrix.duplicated().sum()

0

На всякий случай были проверены типы данных, пропуски и дубликаты, с этим оказалось все в порядке. 

В матрице виднеются нули, есть два варианта развития событий: либо это 0 как оценки пользователей, либо это пропуски, которые на нули были заменены. Так как у нас нет никакой информации об этом, пока принимаем эти 0 за оценки пользователей и не трогаем матрицу.

Перейдем к вопросам по матрице предпочтений.

## <a class="anchor" id="2"><div class="alert alert-block alert-success">Вопрос 1: пересекаются ли 5-ТОР фильмов пользователей Ангелина и пользователя, самого близкого к ней?

In [208]:
user_item_matrix = user_item_matrix.set_index('название фильма')
user_item_matrix

,Алексей,Алена,Алина,Альбина,Анастасия,Ангелина,Андрей,Анна,Арина,Артём,...,Кристина,Майя,Максим,Надежда,Полина,Самира,София,Ульяна,Юлия,Юрий
название фильма,,,,,,,,,,,,,,,,,,,,,
«Дэдпул» (2016),8,7,0,0,0,10,0,8,0,6,...,10,10,10,0,7,0,0,10,0,7
«Бумажный дом» (2019),7,0,0,0,0,10,10,10,0,8,...,10,10,10,0,0,0,0,7,10,7
«Голодные игры» (2012),6,5,0,7,0,10,10,7,0,3,...,0,10,10,0,10,0,0,8,0,8
«Стражи Галактики» (2014),9,8,0,0,0,7,0,10,0,5,...,10,10,10,0,10,0,0,10,0,10
«Интерстеллар» (2014),10,10,0,10,0,10,6,10,0,5,...,0,10,10,0,10,0,0,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Харли Квинн,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,0,8
Слабый герой,0,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0
Мошенники,0,0,0,0,0,0,0,0,0,9,...,10,0,0,0,0,0,0,0,0,0


In [209]:
filtered_matrix1 = user_item_matrix.T
filtered_matrix1

название фильма,«Дэдпул» (2016),«Бумажный дом» (2019),«Голодные игры» (2012),«Стражи Галактики» (2014),«Интерстеллар» (2014),«Социальная сеть» (2010),«Человек-паук: Через вселенные» (2018),«Матрица» (1999),«Матрица. Воскрешение» (2021),«Матрица: Перезагрузка» (2003),...,Игра в кальмара,Черное зеркало,Злой город,Одни из нас,Формула преступления,Харли Квинн,Слабый герой,Мошенники,Укрытие,Дети перемен
Алексей,8,7,6,9,10,9,7,10,10,10,...,8,0,0,0,0,0,0,0,0,0
Алена,7,0,5,8,10,0,0,10,0,0,...,7,0,0,0,0,0,0,0,0,0
Алина,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Альбина,0,0,7,0,10,0,8,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Анастасия,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ангелина,10,10,10,7,10,0,10,10,10,10,...,8,0,0,0,0,0,0,0,0,0
Андрей,0,10,10,0,6,0,0,7,0,0,...,8,0,0,0,0,0,0,0,0,0
Анна,8,10,7,10,10,10,10,10,0,0,...,9,10,0,0,0,0,0,0,0,0
Арина,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Артём,6,8,3,5,5,0,0,9,9,9,...,0,0,0,0,0,0,0,9,0,0


In [210]:
# воспользуемся классом NearestNeighbors для поиска расстояний
knn = NearestNeighbors(metric = 'cosine',
                       algorithm = 'brute',
                       n_neighbors = 20,
                       n_jobs = -1)

# обучим модель
knn.fit(filtered_matrix1)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [211]:
# определимся, сколько рекомендаций мы хотим получить
recommendations = 5

# и для какого зрителя
search_word = 'Ангелина'

In [212]:
# теперь нужно найти индексы и расстояния
# воспользуемся методом kneighbors()
distances, indices = knn.kneighbors(filtered_matrix1.loc[[search_word]], n_neighbors = recommendations + 1)

# Обрабатываем результаты
similar_movies = filtered_matrix1.index[indices.flatten()[1:]]  # Исключаем субъекта рекомендаций
distances = distances.flatten()[1:]

# Создаем DataFrame с результатами
results = pd.DataFrame({
    'Зритель': similar_movies,
    'Расстояние': distances
})
# Добавляем рейтинг (1 - самый близкий)
results['Рейтинг похожести'] = range(1, len(results)+1)

# Сортируем по расстоянию (на всякий случай)
results = results.sort_values('Расстояние')

print(f"Топ-{recommendations} зрителей с интересами, похожими на '{search_word}':\n")
print(results[['Рейтинг похожести', 'Зритель', 'Расстояние']].to_string(index=False))

Топ-5 зрителей с интересами, похожими на 'Ангелина':

 Рейтинг похожести Зритель  Расстояние
                 1    Юрий    0.221743
                 2  Ульяна    0.222368
                 3 Алексей    0.231239
                 4   Игнат    0.233530
                 5 Валерия    0.249075


Получилось, что для пользователя Ангелина самый близкий - Юрий. Посмотрим фильмы для него.

In [213]:
user_item_matrix = pd.read_excel('ratingNames.xlsx')
user_item_matrix

,название фильма,Алексей,Алена,Алина,Альбина,Анастасия,Ангелина,Андрей,Анна,Арина,...,Кристина,Майя,Максим,Надежда,Полина,Самира,София,Ульяна,Юлия,Юрий
0,«Дэдпул» (2016),8,7,0,0,0,10,0,8,0,...,10,10,10,0,7,0,0,10,0,7
1,«Бумажный дом» (2019),7,0,0,0,0,10,10,10,0,...,10,10,10,0,0,0,0,7,10,7
2,«Голодные игры» (2012),6,5,0,7,0,10,10,7,0,...,0,10,10,0,10,0,0,8,0,8
3,«Стражи Галактики» (2014),9,8,0,0,0,7,0,10,0,...,10,10,10,0,10,0,0,10,0,10
4,«Интерстеллар» (2014),10,10,0,10,0,10,6,10,0,...,0,10,10,0,10,0,0,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Харли Квинн,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,0,8
89,Слабый герой,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0
90,Мошенники,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0
91,Укрытие,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0


In [214]:
# находим топ-5 фильмов для Юрия с оценками этих фильмов
search_word = 'Юрий'

In [215]:
a = user_item_matrix.nlargest(5, search_word)[['название фильма', search_word]]
a

,название фильма,Юрий
3,«Стражи Галактики» (2014),10
4,«Интерстеллар» (2014),10
7,«Матрица» (1999),10
16,«Аватар 2» (2022),10
17,«Аватар» (2009),10


In [216]:
# находим топ-5 фильмов для Ангелины с оценками этих фильмов
search_word = 'Ангелина'

In [217]:
a = user_item_matrix.nlargest(5, search_word)[['название фильма', search_word]]
a

,название фильма,Ангелина
0,«Дэдпул» (2016),10
1,«Бумажный дом» (2019),10
2,«Голодные игры» (2012),10
4,«Интерстеллар» (2014),10
6,«Человек-паук: Через вселенные» (2018),10


В пересечении тоже Интерстеллар в фильмах топ-5.

## <a class="anchor" id="2"><div class="alert alert-block alert-success">Вопрос 2: отличается ли средний рейтинг фильмов у пользователей Игнат и похожего на него?

In [218]:
user_item_matrix = user_item_matrix.set_index('название фильма')
user_item_matrix

,Алексей,Алена,Алина,Альбина,Анастасия,Ангелина,Андрей,Анна,Арина,Артём,...,Кристина,Майя,Максим,Надежда,Полина,Самира,София,Ульяна,Юлия,Юрий
название фильма,,,,,,,,,,,,,,,,,,,,,
«Дэдпул» (2016),8,7,0,0,0,10,0,8,0,6,...,10,10,10,0,7,0,0,10,0,7
«Бумажный дом» (2019),7,0,0,0,0,10,10,10,0,8,...,10,10,10,0,0,0,0,7,10,7
«Голодные игры» (2012),6,5,0,7,0,10,10,7,0,3,...,0,10,10,0,10,0,0,8,0,8
«Стражи Галактики» (2014),9,8,0,0,0,7,0,10,0,5,...,10,10,10,0,10,0,0,10,0,10
«Интерстеллар» (2014),10,10,0,10,0,10,6,10,0,5,...,0,10,10,0,10,0,0,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Харли Квинн,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,0,8
Слабый герой,0,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0
Мошенники,0,0,0,0,0,0,0,0,0,9,...,10,0,0,0,0,0,0,0,0,0


In [219]:
filtered_matrix1 = user_item_matrix.T
filtered_matrix1

название фильма,«Дэдпул» (2016),«Бумажный дом» (2019),«Голодные игры» (2012),«Стражи Галактики» (2014),«Интерстеллар» (2014),«Социальная сеть» (2010),«Человек-паук: Через вселенные» (2018),«Матрица» (1999),«Матрица. Воскрешение» (2021),«Матрица: Перезагрузка» (2003),...,Игра в кальмара,Черное зеркало,Злой город,Одни из нас,Формула преступления,Харли Квинн,Слабый герой,Мошенники,Укрытие,Дети перемен
Алексей,8,7,6,9,10,9,7,10,10,10,...,8,0,0,0,0,0,0,0,0,0
Алена,7,0,5,8,10,0,0,10,0,0,...,7,0,0,0,0,0,0,0,0,0
Алина,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Альбина,0,0,7,0,10,0,8,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Анастасия,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ангелина,10,10,10,7,10,0,10,10,10,10,...,8,0,0,0,0,0,0,0,0,0
Андрей,0,10,10,0,6,0,0,7,0,0,...,8,0,0,0,0,0,0,0,0,0
Анна,8,10,7,10,10,10,10,10,0,0,...,9,10,0,0,0,0,0,0,0,0
Арина,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Артём,6,8,3,5,5,0,0,9,9,9,...,0,0,0,0,0,0,0,9,0,0


In [220]:
# воспользуемся классом NearestNeighbors для поиска расстояний
knn = NearestNeighbors(metric = 'cosine',
                       algorithm = 'brute',
                       n_neighbors = 20,
                       n_jobs = -1)

# обучим модель
knn.fit(filtered_matrix1)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [221]:
# определимся, сколько рекомендаций мы хотим получить
recommendations = 5

# и для какого зрителя
search_word = 'Игнат'

In [222]:
# теперь нужно найти индексы и расстояния
# воспользуемся методом kneighbors()
distances, indices = knn.kneighbors(filtered_matrix1.loc[[search_word]], n_neighbors = recommendations + 1)

# Обрабатываем результаты
similar_movies = filtered_matrix1.index[indices.flatten()[1:]]  # Исключаем субъекта рекомендаций
distances = distances.flatten()[1:]

# Создаем DataFrame с результатами
results = pd.DataFrame({
    'Зритель': similar_movies,
    'Расстояние': distances
})
# Добавляем рейтинг (1 - самый близкий)
results['Рейтинг похожести'] = range(1, len(results)+1)

# Сортируем по расстоянию (на всякий случай)
results = results.sort_values('Расстояние')

print(f"Топ-{recommendations} зрителей с интересами, похожими на '{search_word}':\n")
print(results[['Рейтинг похожести', 'Зритель', 'Расстояние']].to_string(index=False))

Топ-5 зрителей с интересами, похожими на 'Игнат':

 Рейтинг похожести  Зритель  Расстояние
                 1     Майя    0.046095
                 2  Дмитрий    0.075094
                 3   Ульяна    0.230282
                 4 Ангелина    0.233530
                 5     Юрий    0.233748


Майя и Игнат близки по интересам, посмотрим их средний рейтинг фильмов

In [223]:
user_ratings = user_item_matrix['Игнат']
print(f"  • Средний рейтинг Игната: {user_ratings.mean():.1f}")

  • Средний рейтинг Игната: 3.2


In [224]:
user_ratings = user_item_matrix['Майя']
print(f"  • Средний рейтинг Майи: {user_ratings.mean():.1f}")

  • Средний рейтинг Майи: 3.1


В итоге их средние рейтинги очень близки, отличаются на 0.1.

## <a class="anchor" id="2"><div class="alert alert-block alert-success">Вопрос 3: отличается ли количество оценок у пользователей Юлия и похожего на него?

In [225]:
# воспользуемся классом NearestNeighbors для поиска расстояний
knn = NearestNeighbors(metric = 'cosine',
                       algorithm = 'brute',
                       n_neighbors = 20,
                       n_jobs = -1)

# обучим модель
knn.fit(filtered_matrix1)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [226]:
# определимся, сколько рекомендаций мы хотим получить
recommendations = 5

# и для какого зрителя
search_word = 'Юлия'

In [227]:
# теперь нужно найти индексы и расстояния
# воспользуемся методом kneighbors()
distances, indices = knn.kneighbors(filtered_matrix1.loc[[search_word]], n_neighbors = recommendations + 1)

# Обрабатываем результаты
similar_movies = filtered_matrix1.index[indices.flatten()[1:]]  # Исключаем субъекта рекомендаций
distances = distances.flatten()[1:]

# Создаем DataFrame с результатами
results = pd.DataFrame({
    'Зритель': similar_movies,
    'Расстояние': distances
})
# Добавляем рейтинг (1 - самый близкий)
results['Рейтинг похожести'] = range(1, len(results)+1)

# Сортируем по расстоянию (на всякий случай)
results = results.sort_values('Расстояние')

print(f"Топ-{recommendations} зрителей с интересами, похожими на '{search_word}':\n")
print(results[['Рейтинг похожести', 'Зритель', 'Расстояние']].to_string(index=False))

Топ-5 зрителей с интересами, похожими на 'Юлия':

 Рейтинг похожести Зритель  Расстояние
                 1  Полина    0.363025
                 2 Алексей    0.365190
                 3    Юрий    0.375473
                 4  Ульяна    0.382730
                 5   Артём    0.388475


Юлия и Полина близки по интересам, посмотрим их средний рейтинг фильмов

In [228]:
user_ratings = user_item_matrix['Игнат']
print(f"  • Оценила фильмов Юлия: {len(user_ratings)}")

  • Оценила фильмов Юлия: 93


In [229]:
user_ratings = user_item_matrix['Полина']
print(f"  • Оценил фильмов Полина: {len(user_ratings)}")

  • Оценил фильмов Полина: 93


Они оценили одинаковое количество фильмов.

## <a class="anchor" id="2"><div class="alert alert-block alert-success">Вопрос 4: отличается ли средний рейтинг фильма «Дэдпул» (2016) и самого похожего на него?

In [230]:
filtered_matrix = user_item_matrix

In [231]:
# воспользуемся классом NearestNeighbors для поиска расстояний
knn = NearestNeighbors(metric = 'cosine',
                       algorithm = 'brute',
                       n_neighbors = 5,
                       n_jobs = -1)

# обучим модель
knn.fit(filtered_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1)

In [232]:
# определимся, сколько рекомендаций мы хотим получить
recommendations = 5

# и на основе какого фильма
search_word = '«Дэдпул» (2016)'

In [233]:
# теперь нужно найти индексы и расстояния фильмов, которые похожи на наш запрос
# воспользуемся методом kneighbors()
distances, indices = knn.kneighbors(filtered_matrix.loc[[search_word]], n_neighbors = recommendations + 1)

In [234]:
indices

array([[ 0,  3, 36, 10,  9, 22]], dtype=int64)

In [235]:
distances

array([[0.        , 0.02319406, 0.10012523, 0.14147297, 0.1480065 ,
        0.15033642]])

In [236]:
# Обрабатываем результаты
similar_movies = filtered_matrix.index[indices.flatten()[1:]]  # Исключаем исходный фильм
distances = distances.flatten()[1:]

# Создаем DataFrame с результатами
results = pd.DataFrame({
    'Фильм': similar_movies,
    'Расстояние': distances
})
# Добавляем рейтинг (1 - самый близкий)
results['Рейтинг похожести'] = range(1, len(results)+1)

# Сортируем по расстоянию (на всякий случай)
results = results.sort_values('Расстояние')

print(f"Топ-{recommendations} похожих фильмов на '{search_word}':\n")
print(results[['Рейтинг похожести', 'Фильм', 'Расстояние']].to_string(index=False))

Топ-5 похожих фильмов на '«Дэдпул» (2016)':

 Рейтинг похожести                              Фильм  Расстояние
                 1          «Стражи Галактики» (2014)    0.023194
                 2 «Стражи Галактики. Часть 2» (2017)    0.100125
                 3        «Матрица: Революция» (2003)    0.141473
                 4     «Матрица: Перезагрузка» (2003)    0.148007
                 5                    «Форсаж» (2001)    0.150336


In [237]:
# загрузим данные
user_item_matrix = pd.read_excel('ratingNames.xlsx')
user_item_matrix

,название фильма,Алексей,Алена,Алина,Альбина,Анастасия,Ангелина,Андрей,Анна,Арина,...,Кристина,Майя,Максим,Надежда,Полина,Самира,София,Ульяна,Юлия,Юрий
0,«Дэдпул» (2016),8,7,0,0,0,10,0,8,0,...,10,10,10,0,7,0,0,10,0,7
1,«Бумажный дом» (2019),7,0,0,0,0,10,10,10,0,...,10,10,10,0,0,0,0,7,10,7
2,«Голодные игры» (2012),6,5,0,7,0,10,10,7,0,...,0,10,10,0,10,0,0,8,0,8
3,«Стражи Галактики» (2014),9,8,0,0,0,7,0,10,0,...,10,10,10,0,10,0,0,10,0,10
4,«Интерстеллар» (2014),10,10,0,10,0,10,6,10,0,...,0,10,10,0,10,0,0,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Харли Квинн,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,0,8
89,Слабый герой,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0
90,Мошенники,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0
91,Укрытие,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0


In [238]:
film = '«Дэдпул» (2016)'
ratings = user_item_matrix.loc[user_item_matrix['название фильма'] == film, users].values[0]
non_zero_ratings = ratings
non_zero_ratings.mean().round(1)

4.4

In [239]:
film = '«Стражи Галактики» (2014)'
ratings = user_item_matrix.loc[user_item_matrix['название фильма'] == film, users].values[0]
non_zero_ratings = ratings
non_zero_ratings.mean().round(1)

4.7

У второго фильма оценка получилась даже больше, чем у первого.

## <a class="anchor" id="2"><div class="alert alert-block alert-success">Вопрос 5: отличается ли количество оценок фильма Укрытие и самого похожего на него?

In [240]:
filtered_matrix = user_item_matrix.set_index('название фильма')

In [241]:
# воспользуемся классом NearestNeighbors для поиска расстояний
knn = NearestNeighbors(metric = 'cosine',
                       algorithm = 'brute',
                       n_neighbors = 5,
                       n_jobs = -1)

# обучим модель
knn.fit(filtered_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1)

In [242]:
# определимся, сколько рекомендаций мы хотим получить
recommendations = 5

# и на основе какого фильма
search_word = 'Укрытие'

In [243]:
# теперь нужно найти индексы и расстояния фильмов, которые похожи на наш запрос
# воспользуемся методом kneighbors()
distances, indices = knn.kneighbors(filtered_matrix.loc[[search_word]], n_neighbors = recommendations + 1)

In [244]:
indices

array([[89, 91, 85, 92, 60, 57]], dtype=int64)

In [245]:
distances

array([[0.        , 0.        , 0.        , 0.        , 0.00496281,
        0.00990099]])

In [246]:
# Обрабатываем результаты
similar_movies = filtered_matrix.index[indices.flatten()[1:]]  # Исключаем исходный фильм
distances = distances.flatten()[1:]

# Создаем DataFrame с результатами
results = pd.DataFrame({
    'Фильм': similar_movies,
    'Расстояние': distances
})
# Добавляем рейтинг (1 - самый близкий)
results['Рейтинг похожести'] = range(1, len(results)+1)

# Сортируем по расстоянию (на всякий случай)
results = results.sort_values('Расстояние')

print(f"Топ-{recommendations} похожих фильмов на '{search_word}':\n")
print(results[['Рейтинг похожести', 'Фильм', 'Расстояние']].to_string(index=False))

Топ-5 похожих фильмов на 'Укрытие':

 Рейтинг похожести                         Фильм  Расстояние
                 1                       Укрытие    0.000000
                 2                    Злой город    0.000000
                 3                  Дети перемен    0.000000
                 4             «Ячейка общества»    0.004963
                 5 «Надо снимать фильмы о любви»    0.009901


In [247]:
# загрузим данные
user_item_matrix = pd.read_excel('ratingNames.xlsx')
user_item_matrix

,название фильма,Алексей,Алена,Алина,Альбина,Анастасия,Ангелина,Андрей,Анна,Арина,...,Кристина,Майя,Максим,Надежда,Полина,Самира,София,Ульяна,Юлия,Юрий
0,«Дэдпул» (2016),8,7,0,0,0,10,0,8,0,...,10,10,10,0,7,0,0,10,0,7
1,«Бумажный дом» (2019),7,0,0,0,0,10,10,10,0,...,10,10,10,0,0,0,0,7,10,7
2,«Голодные игры» (2012),6,5,0,7,0,10,10,7,0,...,0,10,10,0,10,0,0,8,0,8
3,«Стражи Галактики» (2014),9,8,0,0,0,7,0,10,0,...,10,10,10,0,10,0,0,10,0,10
4,«Интерстеллар» (2014),10,10,0,10,0,10,6,10,0,...,0,10,10,0,10,0,0,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Харли Квинн,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,0,8
89,Слабый герой,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0
90,Мошенники,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0
91,Укрытие,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0


In [248]:
film = 'Укрытие'
ratings = user_item_matrix.loc[user_item_matrix['название фильма'] == film, users].values[0]
non_zero_ratings = ratings
len(non_zero_ratings)

30

In [249]:
film = 'Злой город'
ratings = user_item_matrix.loc[user_item_matrix['название фильма'] == film, users].values[0]
non_zero_ratings = ratings
len(non_zero_ratings)

30

У этих фильмов одинаковое количество оценок.

# <div class="alert alert-block alert-info">Дополнительная часть: применим фильтр к матрице предпочтений, чтобы исключить малоактивных пользоваталей и фильмы с малым количеством ненулевых оценок, и ответим на те же самые вопросы.

In [250]:
# Подсчет количества нулей в каждом столбце (кроме первого)
zero_counts = (user_item_matrix == 0).sum()

# Задаем порог
threshold = 85

# Выбираем столбцы, где нулей не больше порога
columns_to_keep = [col for col in user_item_matrix.columns if zero_counts[col] <= threshold]

# Создаем новый DataFrame без "нулевых" столбцов
user_item_matrix1 = user_item_matrix[columns_to_keep]

In [251]:
user_item_matrix1.shape #посмотрим сколько пользователей и фильмов осталось

(93, 21)

In [252]:
filtered_matrix = (
    user_item_matrix1
    .assign(Количество_нулей=lambda x: (x == 0).sum(axis=1))
    .query("Количество_нулей <= 15")
    .drop(columns=['Количество_нулей'])
)
filtered_matrix.shape # посмотрим сколько пользователей и фильмов осталось

(64, 21)

In [253]:
user_item_matrix = filtered_matrix
user_item_matrix

,название фильма,Алексей,Алена,Альбина,Ангелина,Андрей,Анна,Артём,Валерия,Всеволод,...,Дмитрий,Елизавета,Игнат,Кристина,Майя,Максим,Полина,Ульяна,Юлия,Юрий
0,«Дэдпул» (2016),8,7,0,10,0,8,6,8,0,...,10,10,10,10,10,10,7,10,0,7
1,«Бумажный дом» (2019),7,0,0,10,10,10,8,7,0,...,10,10,10,10,10,10,0,7,10,7
2,«Голодные игры» (2012),6,5,7,10,10,7,3,8,7,...,10,10,10,0,10,10,10,8,0,8
3,«Стражи Галактики» (2014),9,8,0,7,0,10,5,7,0,...,10,10,10,10,10,10,10,10,0,10
4,«Интерстеллар» (2014),10,10,10,10,6,10,5,10,10,...,10,10,10,0,10,10,10,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,"""Темный рыцарь""",10,0,0,0,0,10,0,7,10,...,10,0,1,10,0,0,10,0,9,0
79,"""Люди в черном""",10,9,0,7,7,10,10,9,10,...,10,10,1,10,0,0,0,10,0,10
82,Очень странные дела,0,0,0,0,0,10,10,0,10,...,0,0,1,10,10,10,10,7,10,7
83,Игра в кальмара,8,7,0,8,8,9,0,8,0,...,8,0,1,10,10,0,7,8,0,8


## <a class="anchor" id="2"><div class="alert alert-block alert-success">Вопрос 1: пересекаются ли 5-ТОР фильмов пользователей Ангелина и пользователя, самого близкого к ней?

In [271]:
user_item_matrix = user_item_matrix.set_index('название фильма')
user_item_matrix

,Алексей,Алена,Альбина,Ангелина,Андрей,Анна,Артём,Валерия,Всеволод,Георгий,Дмитрий,Елизавета,Игнат,Кристина,Майя,Максим,Полина,Ульяна,Юлия,Юрий
название фильма,,,,,,,,,,,,,,,,,,,,
«Дэдпул» (2016),8,7,0,10,0,8,6,8,0,0,10,10,10,10,10,10,7,10,0,7
«Бумажный дом» (2019),7,0,0,10,10,10,8,7,0,0,10,10,10,10,10,10,0,7,10,7
«Голодные игры» (2012),6,5,7,10,10,7,3,8,7,5,10,10,10,0,10,10,10,8,0,8
«Стражи Галактики» (2014),9,8,0,7,0,10,5,7,0,5,10,10,10,10,10,10,10,10,0,10
«Интерстеллар» (2014),10,10,10,10,6,10,5,10,10,10,10,10,10,0,10,10,10,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"""Темный рыцарь""",10,0,0,0,0,10,0,7,10,0,10,0,1,10,0,0,10,0,9,0
"""Люди в черном""",10,9,0,7,7,10,10,9,10,0,10,10,1,10,0,0,0,10,0,10
Очень странные дела,0,0,0,0,0,10,10,0,10,6,0,0,1,10,10,10,10,7,10,7


In [272]:
filtered_matrix1 = user_item_matrix.T
filtered_matrix1

название фильма,«Дэдпул» (2016),«Бумажный дом» (2019),«Голодные игры» (2012),«Стражи Галактики» (2014),«Интерстеллар» (2014),«Социальная сеть» (2010),«Человек-паук: Через вселенные» (2018),«Матрица» (1999),«Матрица. Воскрешение» (2021),«Матрица: Перезагрузка» (2003),...,«Субстанция»,«Фуриоса: Хроники Безумного Макса»,«Головоломка 2»,"""Оппенгеймер"" (2023)","""Брат 2""","""Темный рыцарь""","""Люди в черном""",Очень странные дела,Игра в кальмара,Черное зеркало
Алексей,8,7,6,9,10,9,7,10,10,10,...,8,0,10,8,9,10,10,0,8,0
Алена,7,0,5,8,10,0,0,10,0,0,...,0,0,0,0,8,0,9,0,7,0
Альбина,0,0,7,0,10,0,8,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ангелина,10,10,10,7,10,0,10,10,10,10,...,5,0,5,0,7,0,7,0,8,0
Андрей,0,10,10,0,6,0,0,7,0,0,...,0,0,10,10,10,0,7,0,8,0
Анна,8,10,7,10,10,10,10,10,0,0,...,0,0,0,10,7,10,10,10,9,10
Артём,6,8,3,5,5,0,0,9,9,9,...,3,10,10,9,9,0,10,10,0,0
Валерия,8,7,8,7,10,4,3,10,8,8,...,8,0,8,0,10,7,9,0,8,0
Всеволод,0,0,7,0,10,10,7,10,0,0,...,8,0,0,10,10,10,10,10,0,10
Георгий,0,0,5,5,10,0,0,8,0,0,...,0,0,0,0,0,0,0,6,0,6


In [273]:
# воспользуемся классом NearestNeighbors для поиска расстояний
knn = NearestNeighbors(metric = 'cosine',
                       algorithm = 'brute',
                       n_neighbors = 20,
                       n_jobs = -1)

# обучим модель
knn.fit(filtered_matrix1)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [274]:
# определимся, сколько рекомендаций мы хотим получить
recommendations = 5

# и для какого зрителя
search_word = 'Ангелина'

In [275]:
# теперь нужно найти индексы и расстояния
# воспользуемся методом kneighbors()
distances, indices = knn.kneighbors(filtered_matrix1.loc[[search_word]], n_neighbors = recommendations + 1)

# Обрабатываем результаты
similar_movies = filtered_matrix1.index[indices.flatten()[1:]]  # Исключаем субъекта рекомендаций
distances = distances.flatten()[1:]

# Создаем DataFrame с результатами
results = pd.DataFrame({
    'Зритель': similar_movies,
    'Расстояние': distances
})
# Добавляем рейтинг (1 - самый близкий)
results['Рейтинг похожести'] = range(1, len(results)+1)

# Сортируем по расстоянию (на всякий случай)
results = results.sort_values('Расстояние')

print(f"Топ-{recommendations} зрителей с интересами, похожими на '{search_word}':\n")
print(results[['Рейтинг похожести', 'Зритель', 'Расстояние']].to_string(index=False))

Топ-5 зрителей с интересами, похожими на 'Ангелина':

 Рейтинг похожести Зритель  Расстояние
                 1  Ульяна    0.203167
                 2    Юрий    0.206692
                 3 Алексей    0.207711
                 4   Игнат    0.226924
                 5 Валерия    0.231438


Получилось, что для пользователя Ангелина самый близкий - Ульяна. Посмотрим фильмы для него.

In [276]:
user_item_matrix = pd.read_excel('ratingNames.xlsx')
user_item_matrix

,название фильма,Алексей,Алена,Алина,Альбина,Анастасия,Ангелина,Андрей,Анна,Арина,...,Кристина,Майя,Максим,Надежда,Полина,Самира,София,Ульяна,Юлия,Юрий
0,«Дэдпул» (2016),8,7,0,0,0,10,0,8,0,...,10,10,10,0,7,0,0,10,0,7
1,«Бумажный дом» (2019),7,0,0,0,0,10,10,10,0,...,10,10,10,0,0,0,0,7,10,7
2,«Голодные игры» (2012),6,5,0,7,0,10,10,7,0,...,0,10,10,0,10,0,0,8,0,8
3,«Стражи Галактики» (2014),9,8,0,0,0,7,0,10,0,...,10,10,10,0,10,0,0,10,0,10
4,«Интерстеллар» (2014),10,10,0,10,0,10,6,10,0,...,0,10,10,0,10,0,0,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Харли Квинн,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,0,8
89,Слабый герой,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0
90,Мошенники,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0
91,Укрытие,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0


In [313]:
# находим топ-5 фильмов для Ульяны с оценками этих фильмов
search_word = 'Ульяна'

In [314]:
a = user_item_matrix.nlargest(5, search_word)[['название фильма', search_word]]
a

,название фильма,Ульяна
0,«Дэдпул» (2016),10
3,«Стражи Галактики» (2014),10
4,«Интерстеллар» (2014),10
7,«Матрица» (1999),10
16,«Аватар 2» (2022),10


In [279]:
# находим топ-5 фильмов для Ангелины с оценками этих фильмов
search_word = 'Ангелина'

In [280]:
a = user_item_matrix.nlargest(5, search_word)[['название фильма', search_word]]
a

,название фильма,Ангелина
0,«Дэдпул» (2016),10
1,«Бумажный дом» (2019),10
2,«Голодные игры» (2012),10
4,«Интерстеллар» (2014),10
6,«Человек-паук: Через вселенные» (2018),10


В пересечении тоже Интерстеллар,  «Дэдпул» (2016) в фильмах топ-5.

## <a class="anchor" id="2"><div class="alert alert-block alert-success">Вопрос 2: отличается ли средний рейтинг фильмов у пользователей Игнат и похожего на него?

In [281]:
user_item_matrix = user_item_matrix.set_index('название фильма')
user_item_matrix

,Алексей,Алена,Алина,Альбина,Анастасия,Ангелина,Андрей,Анна,Арина,Артём,...,Кристина,Майя,Максим,Надежда,Полина,Самира,София,Ульяна,Юлия,Юрий
название фильма,,,,,,,,,,,,,,,,,,,,,
«Дэдпул» (2016),8,7,0,0,0,10,0,8,0,6,...,10,10,10,0,7,0,0,10,0,7
«Бумажный дом» (2019),7,0,0,0,0,10,10,10,0,8,...,10,10,10,0,0,0,0,7,10,7
«Голодные игры» (2012),6,5,0,7,0,10,10,7,0,3,...,0,10,10,0,10,0,0,8,0,8
«Стражи Галактики» (2014),9,8,0,0,0,7,0,10,0,5,...,10,10,10,0,10,0,0,10,0,10
«Интерстеллар» (2014),10,10,0,10,0,10,6,10,0,5,...,0,10,10,0,10,0,0,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Харли Квинн,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,0,8
Слабый герой,0,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0
Мошенники,0,0,0,0,0,0,0,0,0,9,...,10,0,0,0,0,0,0,0,0,0


In [282]:
filtered_matrix1 = user_item_matrix.T
filtered_matrix1

название фильма,«Дэдпул» (2016),«Бумажный дом» (2019),«Голодные игры» (2012),«Стражи Галактики» (2014),«Интерстеллар» (2014),«Социальная сеть» (2010),«Человек-паук: Через вселенные» (2018),«Матрица» (1999),«Матрица. Воскрешение» (2021),«Матрица: Перезагрузка» (2003),...,Игра в кальмара,Черное зеркало,Злой город,Одни из нас,Формула преступления,Харли Квинн,Слабый герой,Мошенники,Укрытие,Дети перемен
Алексей,8,7,6,9,10,9,7,10,10,10,...,8,0,0,0,0,0,0,0,0,0
Алена,7,0,5,8,10,0,0,10,0,0,...,7,0,0,0,0,0,0,0,0,0
Алина,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Альбина,0,0,7,0,10,0,8,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Анастасия,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ангелина,10,10,10,7,10,0,10,10,10,10,...,8,0,0,0,0,0,0,0,0,0
Андрей,0,10,10,0,6,0,0,7,0,0,...,8,0,0,0,0,0,0,0,0,0
Анна,8,10,7,10,10,10,10,10,0,0,...,9,10,0,0,0,0,0,0,0,0
Арина,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Артём,6,8,3,5,5,0,0,9,9,9,...,0,0,0,0,0,0,0,9,0,0


In [283]:
# воспользуемся классом NearestNeighbors для поиска расстояний
knn = NearestNeighbors(metric = 'cosine',
                       algorithm = 'brute',
                       n_neighbors = 20,
                       n_jobs = -1)

# обучим модель
knn.fit(filtered_matrix1)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [284]:
# определимся, сколько рекомендаций мы хотим получить
recommendations = 5

# и для какого зрителя
search_word = 'Игнат'

In [285]:
# теперь нужно найти индексы и расстояния
# воспользуемся методом kneighbors()
distances, indices = knn.kneighbors(filtered_matrix1.loc[[search_word]], n_neighbors = recommendations + 1)

# Обрабатываем результаты
similar_movies = filtered_matrix1.index[indices.flatten()[1:]]  # Исключаем субъекта рекомендаций
distances = distances.flatten()[1:]

# Создаем DataFrame с результатами
results = pd.DataFrame({
    'Зритель': similar_movies,
    'Расстояние': distances
})
# Добавляем рейтинг (1 - самый близкий)
results['Рейтинг похожести'] = range(1, len(results)+1)

# Сортируем по расстоянию (на всякий случай)
results = results.sort_values('Расстояние')

print(f"Топ-{recommendations} зрителей с интересами, похожими на '{search_word}':\n")
print(results[['Рейтинг похожести', 'Зритель', 'Расстояние']].to_string(index=False))

Топ-5 зрителей с интересами, похожими на 'Игнат':

 Рейтинг похожести  Зритель  Расстояние
                 1     Майя    0.046095
                 2  Дмитрий    0.075094
                 3   Ульяна    0.230282
                 4 Ангелина    0.233530
                 5     Юрий    0.233748


Майя и Игнат близки по интересам, посмотрим их средний рейтинг фильмов

In [286]:
user_ratings = user_item_matrix['Игнат']
print(f"  • Средний рейтинг Игната: {user_ratings.mean():.1f}")

  • Средний рейтинг Игната: 3.2


In [287]:
user_ratings = user_item_matrix['Майя']
print(f"  • Средний рейтинг Майи: {user_ratings.mean():.1f}")

  • Средний рейтинг Майи: 3.1


В итоге их средние рейтинги очень близки, отличаются на 0.1.

## <a class="anchor" id="2"><div class="alert alert-block alert-success">Вопрос 3: отличается ли количество оценок у пользователей Юлия и похожего на него?

In [288]:
# воспользуемся классом NearestNeighbors для поиска расстояний
knn = NearestNeighbors(metric = 'cosine',
                       algorithm = 'brute',
                       n_neighbors = 20,
                       n_jobs = -1)

# обучим модель
knn.fit(filtered_matrix1)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [289]:
# определимся, сколько рекомендаций мы хотим получить
recommendations = 5

# и для какого зрителя
search_word = 'Юлия'

In [290]:
# теперь нужно найти индексы и расстояния
# воспользуемся методом kneighbors()
distances, indices = knn.kneighbors(filtered_matrix1.loc[[search_word]], n_neighbors = recommendations + 1)

# Обрабатываем результаты
similar_movies = filtered_matrix1.index[indices.flatten()[1:]]  # Исключаем субъекта рекомендаций
distances = distances.flatten()[1:]

# Создаем DataFrame с результатами
results = pd.DataFrame({
    'Зритель': similar_movies,
    'Расстояние': distances
})
# Добавляем рейтинг (1 - самый близкий)
results['Рейтинг похожести'] = range(1, len(results)+1)

# Сортируем по расстоянию (на всякий случай)
results = results.sort_values('Расстояние')

print(f"Топ-{recommendations} зрителей с интересами, похожими на '{search_word}':\n")
print(results[['Рейтинг похожести', 'Зритель', 'Расстояние']].to_string(index=False))

Топ-5 зрителей с интересами, похожими на 'Юлия':

 Рейтинг похожести Зритель  Расстояние
                 1  Полина    0.363025
                 2 Алексей    0.365190
                 3    Юрий    0.375473
                 4  Ульяна    0.382730
                 5   Артём    0.388475


Юлия и Полина близки по интересам, посмотрим их средний рейтинг фильмов

In [291]:
user_ratings = user_item_matrix['Игнат']
print(f"  • Оценила фильмов Юлия: {len(user_ratings)}")

  • Оценила фильмов Юлия: 93


In [292]:
user_ratings = user_item_matrix['Полина']
print(f"  • Оценила фильмов Полина: {len(user_ratings)}")

  • Оценил фильмов Полина: 93


Они оценили одинаковое количество фильмов.

## <a class="anchor" id="2"><div class="alert alert-block alert-success">Вопрос 4: отличается ли средний рейтинг фильма «Дэдпул» (2016) и самого похожего на него?

In [293]:
filtered_matrix = user_item_matrix

In [294]:
# воспользуемся классом NearestNeighbors для поиска расстояний
knn = NearestNeighbors(metric = 'cosine',
                       algorithm = 'brute',
                       n_neighbors = 5,
                       n_jobs = -1)

# обучим модель
knn.fit(filtered_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1)

In [295]:
# определимся, сколько рекомендаций мы хотим получить
recommendations = 5

# и на основе какого фильма
search_word = '«Дэдпул» (2016)'

In [296]:
# теперь нужно найти индексы и расстояния фильмов, которые похожи на наш запрос
# воспользуемся методом kneighbors()
distances, indices = knn.kneighbors(filtered_matrix.loc[[search_word]], n_neighbors = recommendations + 1)

In [297]:
indices

array([[ 0,  3, 36, 10,  9, 22]], dtype=int64)

In [298]:
distances

array([[0.        , 0.02319406, 0.10012523, 0.14147297, 0.1480065 ,
        0.15033642]])

In [299]:
# Обрабатываем результаты
similar_movies = filtered_matrix.index[indices.flatten()[1:]]  # Исключаем исходный фильм
distances = distances.flatten()[1:]

# Создаем DataFrame с результатами
results = pd.DataFrame({
    'Фильм': similar_movies,
    'Расстояние': distances
})
# Добавляем рейтинг (1 - самый близкий)
results['Рейтинг похожести'] = range(1, len(results)+1)

# Сортируем по расстоянию (на всякий случай)
results = results.sort_values('Расстояние')

print(f"Топ-{recommendations} похожих фильмов на '{search_word}':\n")
print(results[['Рейтинг похожести', 'Фильм', 'Расстояние']].to_string(index=False))

Топ-5 похожих фильмов на '«Дэдпул» (2016)':

 Рейтинг похожести                              Фильм  Расстояние
                 1          «Стражи Галактики» (2014)    0.023194
                 2 «Стражи Галактики. Часть 2» (2017)    0.100125
                 3        «Матрица: Революция» (2003)    0.141473
                 4     «Матрица: Перезагрузка» (2003)    0.148007
                 5                    «Форсаж» (2001)    0.150336


In [300]:
# загрузим данные
user_item_matrix = pd.read_excel('ratingNames.xlsx')
user_item_matrix

,название фильма,Алексей,Алена,Алина,Альбина,Анастасия,Ангелина,Андрей,Анна,Арина,...,Кристина,Майя,Максим,Надежда,Полина,Самира,София,Ульяна,Юлия,Юрий
0,«Дэдпул» (2016),8,7,0,0,0,10,0,8,0,...,10,10,10,0,7,0,0,10,0,7
1,«Бумажный дом» (2019),7,0,0,0,0,10,10,10,0,...,10,10,10,0,0,0,0,7,10,7
2,«Голодные игры» (2012),6,5,0,7,0,10,10,7,0,...,0,10,10,0,10,0,0,8,0,8
3,«Стражи Галактики» (2014),9,8,0,0,0,7,0,10,0,...,10,10,10,0,10,0,0,10,0,10
4,«Интерстеллар» (2014),10,10,0,10,0,10,6,10,0,...,0,10,10,0,10,0,0,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Харли Квинн,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,0,8
89,Слабый герой,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0
90,Мошенники,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0
91,Укрытие,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0


In [301]:
film = '«Дэдпул» (2016)'
ratings = user_item_matrix.loc[user_item_matrix['название фильма'] == film, users].values[0]
non_zero_ratings = ratings
non_zero_ratings.mean().round(1)

6.6

In [302]:
film = '«Стражи Галактики» (2014)'
ratings = user_item_matrix.loc[user_item_matrix['название фильма'] == film, users].values[0]
non_zero_ratings = ratings
non_zero_ratings.mean().round(1)

7.0

У второго фильма оценка получилась даже больше, чем у первого.

## <a class="anchor" id="2"><div class="alert alert-block alert-success">Вопрос 5: отличается ли количество оценок фильма Укрытие и самого похожего на него?

In [303]:
filtered_matrix = user_item_matrix.set_index('название фильма')

In [304]:
# воспользуемся классом NearestNeighbors для поиска расстояний
knn = NearestNeighbors(metric = 'cosine',
                       algorithm = 'brute',
                       n_neighbors = 5,
                       n_jobs = -1)

# обучим модель
knn.fit(filtered_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1)

In [305]:
# определимся, сколько рекомендаций мы хотим получить
recommendations = 5

# и на основе какого фильма
search_word = 'Укрытие'

In [306]:
# теперь нужно найти индексы и расстояния фильмов, которые похожи на наш запрос
# воспользуемся методом kneighbors()
distances, indices = knn.kneighbors(filtered_matrix.loc[[search_word]], n_neighbors = recommendations + 1)

In [307]:
indices

array([[89, 91, 85, 92, 60, 57]], dtype=int64)

In [308]:
distances

array([[0.        , 0.        , 0.        , 0.        , 0.00496281,
        0.00990099]])

In [309]:
# Обрабатываем результаты
similar_movies = filtered_matrix.index[indices.flatten()[1:]]  # Исключаем исходный фильм
distances = distances.flatten()[1:]

# Создаем DataFrame с результатами
results = pd.DataFrame({
    'Фильм': similar_movies,
    'Расстояние': distances
})
# Добавляем рейтинг (1 - самый близкий)
results['Рейтинг похожести'] = range(1, len(results)+1)

# Сортируем по расстоянию (на всякий случай)
results = results.sort_values('Расстояние')

print(f"Топ-{recommendations} похожих фильмов на '{search_word}':\n")
print(results[['Рейтинг похожести', 'Фильм', 'Расстояние']].to_string(index=False))

Топ-5 похожих фильмов на 'Укрытие':

 Рейтинг похожести                         Фильм  Расстояние
                 1                       Укрытие    0.000000
                 2                    Злой город    0.000000
                 3                  Дети перемен    0.000000
                 4             «Ячейка общества»    0.004963
                 5 «Надо снимать фильмы о любви»    0.009901


In [310]:
# загрузим данные
user_item_matrix = pd.read_excel('ratingNames.xlsx')
user_item_matrix

,название фильма,Алексей,Алена,Алина,Альбина,Анастасия,Ангелина,Андрей,Анна,Арина,...,Кристина,Майя,Максим,Надежда,Полина,Самира,София,Ульяна,Юлия,Юрий
0,«Дэдпул» (2016),8,7,0,0,0,10,0,8,0,...,10,10,10,0,7,0,0,10,0,7
1,«Бумажный дом» (2019),7,0,0,0,0,10,10,10,0,...,10,10,10,0,0,0,0,7,10,7
2,«Голодные игры» (2012),6,5,0,7,0,10,10,7,0,...,0,10,10,0,10,0,0,8,0,8
3,«Стражи Галактики» (2014),9,8,0,0,0,7,0,10,0,...,10,10,10,0,10,0,0,10,0,10
4,«Интерстеллар» (2014),10,10,0,10,0,10,6,10,0,...,0,10,10,0,10,0,0,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Харли Квинн,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,0,8
89,Слабый герой,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0
90,Мошенники,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0
91,Укрытие,0,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,0,0,0,0


In [311]:
film = 'Укрытие'
ratings = user_item_matrix.loc[user_item_matrix['название фильма'] == film, users].values[0]
non_zero_ratings = ratings
len(non_zero_ratings)

20

In [312]:
film = 'Злой город'
ratings = user_item_matrix.loc[user_item_matrix['название фильма'] == film, users].values[0]
non_zero_ratings = ratings
len(non_zero_ratings)

20

У этих фильмов одинаковое количество оценок.